In [1]:
import collections
import csv
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import CommonData.notebookutil as nbu
import sys
sys.meta_path.append(nbu.NotebookFinder())

In [3]:
import Kdtree as kdtree
import Clustering as clustering
import Utility.Utility as utility
import ReconstructSpectra as reconstruction

In [21]:
initial_xy_data = utility.load_spectra("CommonData/xyz_m.csv", offset=0)
spectra_data = utility.load_spectra("CommonData/spectrum_m.csv", offset=1)
xy_data = clustering.initialize_xy_data(initial_xy_data)

In [14]:
current_tree = [
    [0.332943171211674, 1, 0],
    [0.389574245, 0, 0]
]
for num, i in enumerate(current_tree):
    xy_data = clustering.prepare_cluster_table(xy_data, i, num + 1)

In [15]:
# visualize the clustering
color = matplotlib.colors.cnames.values()
cluster_counter = collections.Counter()
for i in xy_data:
    cluster_counter[int(i[3])] += 1
#    plt.scatter(i[1], i[2], c=color[int(i[3])])
plt.show()
print cluster_counter

next_cluster_index = int(max(xy_data[:, 3])) + 1 
print "next cluster index", next_cluster_index

Counter({1: 456, 0: 374, 2: 46})
next cluster index 3


In [19]:
wfile = open("EvaluateClustering" + str(next_cluster_index+1) + ".csv", "wb")
writer = csv.writer(wfile)
writer.writerow(["number of clusters",
                 "average error",
                 "max error",
                 "min error",
                 "median error",
                 "split value",
                 "split dimension", 
                 "split target cluster"])
wfile.close()

In [20]:
for split_dim in [0,1]:#0->split xaxis, 1->split yaxis 
    for target_cluster in range(next_cluster_index): #split target cluster
        print split_dim, target_cluster, time.ctime()
        for split_index in range(xy_data.shape[0]): #index of slice key data
            new_tree_key = clustering.split_and_clustering(xy_data, spectra_data, target_cluster, 
                                            split_index, split_dim, next_cluster_index, current_tree)
            if(new_tree_key != False):
                pc_list, mean_list = reconstruction.load_pc_mean_multi("PCAresult/componentsPCAcluss", 
                                                                        "PCAresult/meanPCAcluss", 
                                                                        next_cluster_index + 1)    
                new_tree = kdtree.make_tree(new_tree_key)
                test_spectra = utility.load_spectra("CommonData/spectrum_m.csv")
                test_reconstruct_spectra = reconstruction.reconstruct_spectra(pc_list, mean_list, test_spectra, next_cluster_index, new_tree)
                if(utility.output_flag):
                    utility.write_result_with_header(test_reconstruct_spectra, "reconstructSpectrum.csv", 
                                                  range(test_reconstruct_spectra.shape[0]))
                error_list = reconstruction.calc_err(test_reconstruct_spectra, test_spectra)
                reconstruction.write_score(next_cluster_index+1, error_list, new_tree_key, 
                                           "EvaluateClustering" + str(next_cluster_index+1) + ".csv")
print "fin", time.ctime()

0 0 Fri Jun 22 01:34:39 2018


KeyboardInterrupt: 